# Snow Depth - ICESat-2 Applications Tutorial

```{admonition} Learning Objectives
**This is a placeholder Notebook for the snow depth tutorial**
- gain experience in working with SlideRule to access and pre-process ICESat-2 data
- learn how use projections and interpolation to compare ICESat-2 track data with gridded raster products
- develop a general understanding of how to measure snow depths with LiDAR, and learn about opportunities and challenges when using ICEsat-2 along-track products
```

## Computing environment

We'll be using the following open source Python libraries in this notebook:

In [ ]:
import ipyleaflet
from ipyleaflet import Map, Rectangle, basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

import ipywidgets
import datetime
import re

# maybe the ones below??
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import CRS
from sliderule import sliderule, icesat2, earthdata, h5

## Draft outline:

Background:
- How do we measure snow depth with lidar
- How do we measure snow depth with ICESat-2 specifically
- What challenges do we need to think about
    - Comparing a raster to an IS-2 transect
    - Geolocation
    - Vegetation, slope affects
- What do we need
    - A region of interest
    - ICESat-2 data
    - A snow-free reference DEM
        
Define ROI:
- Define bounding box
- Plot box over satellite image? Map showing location of ROI?
    
Calculate ATL06:
- Talk a little about why were using ALT06 (presuming we decide to use ATL06)
- Set Parameters
    - Talk a little about the parameters, provide link to more detailed info from SlideRule website
- Show code cell to calculate ICESat-2 data for demonstration
- Read in previously calculated data
- Plot ICESat-2 tracks on ROI, height colorbar?
    
Pull DEM:
- Talk about raster data?
- Show code cell to pull 3DEP data for demonstration purposes
- Read in previously downloaded DEM
- Plot
    
Compare IS-2 and DEM:
	- Remind audience that we need to process the raster data to be comparable to a point transect
	- Integrate DEM (mention that there are a number of ways to do this, we are showing a simple way for time)
	- Plot snow depths

## Data

Use SlideRule to acquire ATL06 data as well as the gridded data, noting customization for averaging footprint, photon identification: this will be a modified ATL06 download that includes some vegetation filtering


**Data considerations:**

pre-download the demonstration files to the CryoHub checking in on maximum file size - try to keep the region of interest relatively small for the tutorial? @zachghiaccio to contact CryoHub folks
checking to see if SlideRule can be used to acquire the gridded DEM @KarinaZikan.

In [ ]:
# Initialize SlideRule
icesat2.init("slideruleearth.io")

In [ ]:
# Define region of interest over ACP, Alaska
region = [ {"lon":-148.85, "lat": 69.985},
           {"lon":-148.527, "lat": 69.985},
           {"lon":-148.527, "lat": 70.111},
           {"lon":-148.85, "lat": 70.111},
           {"lon":-148.85, "lat": 69.985} ]

# Alternate method, with geoJSON
#path = '/home/jovyan/ICESat-2-Hackweek-2023/book/tutorials/applications-snow-depth/'
#region = icesat2.toregion(f'{path}acp_lidar_box.geojson')["poly"]

In [ ]:
# Build SlideRule request
parms = {
    "poly": region,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "atl08_class": ["atl08_ground"],
    "ats": 5.0,
    "len": 20.0,
    "res": 10.0,
    "maxi": 5
}

is2_df = icesat2.atl06p(parms, "nsidc-s3")

is2_df.head()

## sample the DTM to ICESat-2 ground track 
mention importance of geolocation errors and choice of interpolation

## differencing to get a snowdepth


## visualize the result

## Interactive visualization

The ipyleaflet library allows us to create interactive map visualizations. Below we define a geographic bounding box for our area of interest, and plot it on an interactive "slippy map".

In [ ]:
bbox = [-108.3, 39.2, -107.8, 38.8]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

Display the bounding box on an interactive basemap for context. All the available basemaps can be found in the [ipyleaflet documentation](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)

In [ ]:
m = Map(center=bbox_ctr, zoom=10)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

### NASA GIBS basemap

NASA's [Global Imagery Browse Services (GIBS)](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs) is a great Web Map Tile Service (WMTS) to visualize NASA data as pre-rendered tiled raster images. The NASA [Worldview](https://worldview.earthdata.nasa.gov) web application is a way to explore all GIBS datasets. We can also use ipyleaflet to explore GIBS datasets, like MODIS truecolor images, within a Jupyter Notebook. Use the slider in the image below to reveal the image from 2019-04-25:

In [ ]:
m = Map(center=bbox_ctr, zoom=6)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-04-25")
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

m

### Exercise 1

Re-create the map above using different tile layers for both the right and left columns. Label a single point of interest with a marker, such as the city of Grand Junction, Colorado.

In [ ]:
# Add your solution for exercise 1 here!

## Summary

 🎉 Congratulations! You've completely this tutorial and have seen how we can add  notebook can be formatted, and how to create interactive map visualization with ipyleaflet.
 

```{note}
You may have noticed Jupyter Book adds some extra formatting features that do not necessarily render as you might expect when *executing* a noteook in Jupyter Lab. This "admonition" note is one such example.
```

:::{warning}
Jupyter Book is very particular about [Markdown header ordering](https://jupyterbook.org/structure/sections-headers.html?highlight=headers#how-headers-and-sections-map-onto-to-book-structure) to automatically create table of contents on the website. In this tutorial we are careful to use a single main header (#) and sequential subheaders (#, ##, ###, etc.)
:::

## References

To further explore the topics of this tutorial see the following detailed documentation:

* [Jupyter Book rendering of .ipynb notebooks](https://jupyterbook.org/file-types/notebooks.html)
* [Jupyter Book guide on writing narrative content](https://jupyterbook.org/content/index.html)
* [ipyleaflet documentation](https://ipyleaflet.readthedocs.io)